In [ ]:
from transformers import pipeline
import gradio as gr

classify_pipeline = pipeline(task="text-classification", model="jonmanly/custServiceClassifier",return_all_scores=True)
sentiment_pipeline = pipeline("sentiment-analysis",model="distilbert-base-uncased-finetuned-sst-2-english")

defaultQuery = "I've heard great things about you guys.  I'd like to open an account."

def classify(input_txt):
    predictions = classify_pipeline(input_txt)
    allPreds = [[x['label'],round(x['score'],3)] for x in predictions[0]]
    allPreds.sort(key=lambda x: x[1],reverse=True)
    answer='\n'.join([x[0]+": "+str(x[1]) for x in allPreds[:3]])
    return answer

def sentimentScore(input_txt):
    sentiment = sentiment_pipeline(input_txt)[0]
    sentiment_statement = "Sentiment is "+sentiment['label']+": "+str(round(sentiment['score'],3))
    return sentiment_statement

def classifyAndSentiment(input_txt):
    classifyResp = classify(input_txt)
    sentimentResp = sentimentScore(input_txt)
    resp = f"{classifyResp}\n\n{sentimentResp}"
    return resp

gradio_app = gr.Interface(
    classifyAndSentiment,
    inputs=gr.Text(label="Say what you want here to the Customer Service Classifier",autofocus=True,placeholder=defaultQuery ),
    outputs=gr.Text(label="Top 3 Classifications and Probability Scores"),
    title="Customer Service Classifier Demo",
)

if __name__ == "__main__":
    gradio_app.launch()

In [ ]:
from transformers import pipeline





In [ ]:

sentiment_statement = "Sentiment is "+sentiment[0]['label']+": "+str(round(sentiment[0]['score'],3))
sentiment_statement

In [ ]:
sentiment

In [ ]:
sentiment_pipeline = pipeline("sentiment-analysis")
input_txt="give me back my money!"
sentiment = sentiment_pipeline(input_txt)[0]
sentiment_statement = "Sentiment is "+sentiment['label']+": "+str(round(sentiment['score'],3))

In [ ]:
sentiment

In [ ]:
sentiment_pipeline